## LLM Experimentation with DB + LangChain

In [1]:
# !pip install langchain -q
# !pip install langchain-experimental -q
# !pip install boto3 --upgrade -q
!pip install s3fs --upgrade -q
!pip install sqlalchemy-redshift psycopg2-binary -q --upgrade
!pip install sqlalchemy --upgrade -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.80 requires botocore==1.31.80, but you have botocore 1.31.64 which is incompatible.
boto3 1.28.80 requires botocore<1.32.0,>=1.31.80, but you have botocore 1.31.64 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy-redshift 0.8.14 requires SQLAlchemy<2.0.0,>=0.9.2, but you have sqlalchemy 2.0.23 which is incompatible.


In [2]:
import re
import boto3
import pandas as pd
import os
import sqlite3

from tqdm import tqdm
tqdm.pandas()

from IPython.display import display, HTML

from langchain.llms.bedrock import Bedrock

from langchain.prompts import PromptTemplate
from langchain.utilities import SQLDatabase

from langchain.chains import create_sql_query_chain

from prompts import *

In [3]:
QA_DATA_PATH = "s3://aws-gaiic-merck-text2sql/data/synpuf_query_combined_20231121.csv"
COLUMNS_DESC_PATH = '../data/columns_description.xml'

QUESTION = 'Count the total number of inpatients'

MODEL_ID = 'anthropic.claude-v2'
MODEL_CONFIG = {
    "max_tokens_to_sample": 512,
    "temperature": 0,
    "top_p": 1,
    "top_k": 50,
}

LOAD_OUTPUTS = False

PROMPT_VERSION = 'V2'
PROMPT_SQLITE = PROMPT_V2_SQLITE
PROMPT_REDSHIFT = PROMPT_V2_REDSHIFT

OUTPUT_PATH = f'../outputs/llms/prompt_{PROMPT_VERSION}.csv'

#Redshift params
REDSHIFT_HOST = "redshift-synpuf-db-cluster-1.ckbiikj73zqo.us-east-1.redshift.amazonaws.com"
REDSHIFT_PORT = 5439
REDSHIFT_DATABASE = 'dev'
USERNAME = "awsuser"
PASSWORD = "Merck123"
DB_SCHEMA = 'rwdex_raw_synpuf'

REDSHIFT_ENDPOINT = f"redshift+psycopg2://{USERNAME}:{PASSWORD}@{REDSHIFT_HOST}:{REDSHIFT_PORT}/{REDSHIFT_DATABASE}"

In [4]:
bedrock_runtime = boto3.client('bedrock-runtime')

def get_bedrock_llm(model_id, model_config, verbose=False, bedrock_client=bedrock_runtime):
    llm = Bedrock(client=bedrock_runtime,
                  model_id=model_id, 
                  verbose=verbose,
                  model_kwargs=model_config)
    return llm


def extract_xml(tags, text):
    """Extract content enclosed in XML tags and remove newline symbols within the tag."""
    # First, extract content between the tags
    for tag in tags:
        xml_pattern = f"(?:<{tag}.*?>)(.*?)(?:<\\/{tag}>)"
        extracted_content = re.findall(xml_pattern, text, re.DOTALL)
        
        if extracted_content:
            # Then, remove newline symbols from the extracted content
            return extracted_content[0].replace('\n', ' ').strip()

    return None


def get_sql(row0, chain, db_engine='sqlite'):
    row = row0.copy()
    
    gt_exec_col = "query_corrected_with_list_executed"
    if db_engine =='sqlite':
        gt_exec_col = "query_corrected_sqlite_with_list_executed"
    
    question = row['question_with_list']
    
    sql = 'GT SQL Non-Executable'
    if row[gt_exec_col]:
        response = db_chain.invoke({"question": question})
        sql = extract_xml(tags=['sql'], text=response)
    
    row[f'sql_predicted_{db_engine}'] = sql
    return row


def print_gt_and_pred_sql(row, gt_column, pred_column):
    """Print both the ground-truth and predicted SQL queries."""

    gt_sql = row[gt_column]
    if 'sqlite' in gt_column:
        gt_sql = gt_sql.replace('rwdex_raw_synpuf.', '')

    print(f"QA ID: {row['qa_id']}")
    print('-'*30)

    display(HTML(f"<font color=blue>QUESTION: {row['question_with_list']}</font>"))
    print('-'*30)
    
    display(HTML(f"<font color=green>SQL(GT): {gt_sql}</font>"))
    print('-'*30)

    display(HTML(f"<font color=gray>SQL(PRED): {row[pred_column]}</font>"))
    print('*'*30)

In [5]:
#Load Column Descriptions
with open(COLUMNS_DESC_PATH,'r') as f:
    col_descriptions = f.readlines()[1:]
col_descriptions = ''.join(col_descriptions)
print(col_descriptions[:500])

<data>
  <row>
    <column_name>DESYNPUF_ID</column_name>
    <column_description>DESYNPUF: Beneficiary Code</column_description>
    <table_name>beneficiary_summary</table_name>
  </row>
  <row>
    <column_name>BENE_BIRTH_DT</column_name>
    <column_description>DESYNPUF: Date of birth</column_description>
    <table_name>beneficiary_summary</table_name>
  </row>
  <row>
    <column_name>BENE_DEATH_DT</column_name>
    <column_description>DESYNPUF: Date of death</column_description>
    <table


In [6]:
db = SQLDatabase.from_uri(REDSHIFT_ENDPOINT,  schema=f'{DB_SCHEMA}')#engine_args={'options': '-csearch_path={}'.format(DB_SCHEMA)})
llm = get_bedrock_llm(model_id=MODEL_ID, verbose=True, model_config=MODEL_CONFIG)

In [7]:
#Get table names in DB
# db.get_usable_table_names()

In [8]:
# db.table_info

In [9]:
prompt = PROMPT_REDSHIFT.replace('{db_schema}', DB_SCHEMA)
prompt = prompt.replace('{columns_description}', col_descriptions)
prompt = PromptTemplate.from_template(prompt)
db_chain = create_sql_query_chain(llm, db, prompt=prompt)

In [10]:
QUESTION = "select all inpatients in which their claim started in 2005"
response = db_chain.invoke({"question": QUESTION, 'db_schema': DB_SCHEMA})
print(response)

<sql>
SELECT DISTINCT "DESYNPUF_ID"
FROM rwdex_raw_synpuf."inpatient_claims"
WHERE "CLM_FROM_DT" BETWEEN '2005-01-01' AND '2005-12-31'
</sql>


In [11]:
sql = extract_xml(tags=['sql'], text=response)
print(sql)

SELECT DISTINCT "DESYNPUF_ID" FROM rwdex_raw_synpuf."inpatient_claims" WHERE "CLM_FROM_DT" BETWEEN '2005-01-01' AND '2005-12-31'


### Run for all examples

In [12]:
df_qa = pd.read_csv(QA_DATA_PATH)
print(df_qa.shape)
df_qa.head()

(40, 13)


qa_id                                 question  \
0    t1          count the number of outpatients   
1    t2           count the number of inpatients   
2    t3       select all inpatients in year 2005   
3    t4      select all outpatients in year 2006   
4    t5  return total patients from 2015 to 2016   

                        question_with_list  \
0          count the number of outpatients   
1           count the number of inpatients   
2       select all inpatients in year 2005   
3      select all outpatients in year 2006   
4  return total patients from 2015 to 2016   

                                               query  \
0  select count(distinct desynpuf_id) from rwdex_...   
1  select count(distinct desynpuf_id) from rwdex_...   
2  select distinct desynpuf_id from rwdex_raw_syn...   
3  select distinct desynpuf_id from rwdex_raw_syn...   
4  select distinct desynpuf_id from rwdex_raw_syn...   

                                     query_corrected  \
0  select count(distinct desynpuf_id) from rwdex_...   
1  select count(distinct desynpuf_id) from rwdex_...   
2  select distinct desynpuf_id from rwdex_raw_syn...   
3  select distinct desynpuf_id from rwdex_raw_syn...   
4  select distinct desynpuf_id from rwdex_raw_syn...   

                           query_corrected_with_list  difficulty_level  \
0  select count(distinct desynpuf_id) from rwdex_...                 1   
1  select count(distinct desynpuf_id) from rwdex_...                 1   
2  select distinct desynpuf_id from rwdex_raw_syn...                 2   
3  select distinct desynpuf_id from rwdex_raw_syn...                 2   
4  select distinct desynpuf_id from rwdex_raw_syn...                 3   

  code_list_needed code_list_table  \
0              NaN             NaN   
1              NaN             NaN   
2              Yes             NaN   
3              Yes             NaN   
4              NaN             NaN   

                                       gpt4_response  \
0  SELECT COUNT(*) AS outpatient_count\nFROM rwde...   
1  SELECT COUNT(DISTINCT desynpuf_id) AS distinct...   
2  SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_sy...   
3  SELECT DISTINCT desynpuf_id\nFROM rwdex_raw_sy...   
4  SELECT COUNT(DISTINCT desynpuf_id) AS total_pa...   

                                 gpt4_followup_input  \
0            I want the distinct numbers of patients   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  I want results from two tables: outpatient and...   

                               gpt4_revised_response  \
0  SELECT COUNT(DISTINCT desynpuf_id) AS distinct...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  SELECT 'Outpatient' AS patient_type, COUNT(DIS...   

   query_corrected_with_list_executed  
0                                True  
1                                True  
2                                True  
3                                True  
4                                True

In [13]:
# Count number of executable SQLs
df_qa.query_corrected_with_list_executed.value_counts()

True     30
False    10
Name: query_corrected_with_list_executed, dtype: int64

In [14]:
if LOAD_OUTPUTS:
    df_qa = pd.read_csv(OUTPUT_PATH)
else:
    df_qa = df_qa.progress_apply(get_sql, args=(db_chain, 'redshift'), axis=1)

100%|██████████| 40/40 [08:34<00:00, 12.87s/it]


In [15]:
#Display GT and predicted SQL
_ = df_qa.apply(print_gt_and_pred_sql, args=("query_corrected_with_list", "sql_predicted_redshift"), axis=1)

QA ID: t1
------------------------------


------------------------------


------------------------------


******************************
QA ID: t2
------------------------------


------------------------------


...

------------------------------


******************************
QA ID: t40
------------------------------


------------------------------


------------------------------


******************************


In [16]:
## Save Outputs
df_qa.to_csv(OUTPUT_PATH, index=False)

<table>
    <bene_summary>
<columns>
    